<center><em>Courtesy:KGPTalkie</em></center>

- The user can enter a question, and the code retrieves the most similar questions from the dataset using the util.semantic_search method.

- This is a symmetric search task, as the search queries have the same length and content as the questions in the corpus.

- Quora Questions Dataset: https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs



- As embeddings model, we use the SBERT model `quora-distilbert-multilingual`,
that it aligned for 100 languages. I.e., you can type in a question in various languages and it will return the closest questions in the corpus (questions in the corpus are mainly in English).

In [3]:
# https://pytorch.org/get-started/locally/
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


                                              0.0/1.2 MB ? eta -:--:--
                                              0.0/1.2 MB ? eta -:--:--
                                              0.0/1.2 MB ? eta -:--:--
     -                                        0.0/1.2 MB 325.1 kB/s eta 0:00:04
     -                                        0.0/1.2 MB 325.1 kB/s eta 0:00:04
     -                                        0.0/1.2 MB 325.1 kB/s eta 0:00:04
     -                                        0.0/1.2 MB 130.7 kB/s eta 0:00:09
     --                                       0.1/1.2 MB 181.6 kB/s eta 0:00:07
     --                                       0.1/1.2 MB 229.0 kB/s eta 0:00:05
     ---                                      0.1/1.2 MB 273.1 kB/s eta 0:00:04
     ---                                      0.1/1.2 MB 273.1 kB/s eta 0:00:04
     ---                                      0.1/1.2 MB 273.1 kB/s eta 0:00:04
     ----                                     0.1/1.2 MB 232.2 kB/

In [4]:
#algorithm is for fine tuned question retrieval
#pytorch==> when dealing with local system
import torch
torch.__version__

'2.2.1+cpu'

In [5]:
from sentence_transformers import SentenceTransformer, util
import os
import time
import pandas as pd

In [6]:
df = pd.read_csv(r'C:\Users\mmanasanjani\Desktop\Extra\MK\AMLDLP\quora_duplicate_questions.tsv', sep='\t', nrows=20000)

In [7]:
df.shape

(20000, 6)

In [8]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
#mixing question1 and question2 to get 40000 unique questions:
corpus_senteces = list(set(df['question1'].tolist() + df['question2'].tolist())) #set--> to get unique 
len(corpus_senteces)

37766

In [12]:
model = SentenceTransformer('quora-distilbert-multilingual') #search questions in other languages too

pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
corpus_embeddings = model.encode(corpus_senteces, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/1181 [00:00<?, ?it/s]

In [14]:
model._target_device

`SentenceTransformer._target_device` has been removed, please use `SentenceTransformer.device` instead.


device(type='cpu')

In [16]:
corpus_embeddings = corpus_embeddings.to(model._target_device)

`SentenceTransformer._target_device` has been removed, please use `SentenceTransformer.device` instead.


In [18]:
while True:
    query = input("Please enter a question: ")

    if query == 'n':
        break

    question_embedding = model.encode(query, convert_to_tensor=True) #need to project query into same vector space to find similar questions
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=5) #semnatic search functionality-query,corpus embeddings, top5 results
    hits = hits[0]
    print(hits)

    for hit in hits:
        print(hit['score'], ": ", corpus_senteces[hit['corpus_id']])

    print('\n\n\n\n')

Please enter a question: what is python
[{'corpus_id': 3036, 'score': 0.9261703491210938}, {'corpus_id': 34677, 'score': 0.9184613227844238}, {'corpus_id': 3122, 'score': 0.9140185117721558}, {'corpus_id': 17883, 'score': 0.913104236125946}, {'corpus_id': 16515, 'score': 0.9106531143188477}]
0.9261703491210938 :  What are some cool python scripts?
0.9184613227844238 :  What is the difference between () and [] in Python?
0.9140185117721558 :  What are the differences between Python 1, 2, 3?
0.913104236125946 :  The world's largest dinosaur discovered is a titanosaur. How big was his poop?
0.9106531143188477 :  What is the best beginner friendly book on python?





Please enter a question: how to learn python
[{'corpus_id': 3425, 'score': 0.9927082061767578}, {'corpus_id': 34979, 'score': 0.9891511797904968}, {'corpus_id': 26144, 'score': 0.9827980995178223}, {'corpus_id': 4060, 'score': 0.963792085647583}, {'corpus_id': 22154, 'score': 0.9635708332061768}]
0.9927082061767578 :  What ar